# Egortrushin 2-Tree Base Configuration Approach

Implement the egortrushin lattice approach:
1. Start with 2 base trees in specific configuration
2. Translate in x and y directions using nt=[nx, ny] to create nx*ny trees
3. Optimize base configuration AND translation vectors jointly using SA

In [1]:
import numpy as np
import pandas as pd
import random
import copy
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.strtree import STRtree
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 25
scale_factor = Decimal("1e15")

# Tree geometry (15 vertices)
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
TREE_COORDS = list(zip(TX, TY))

print("Setup complete.")

Setup complete.


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""
    
    def __init__(self, center_x=0, center_y=0, angle=0):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._update_polygon()
    
    def _update_polygon(self):
        poly = Polygon(TREE_COORDS)
        poly = rotate(poly, self.angle, origin=(0, 0))
        poly = translate(poly, self.center_x, self.center_y)
        self.polygon = poly
    
    def get_params(self):
        return self.center_x, self.center_y, self.angle
    
    def set_params(self, x, y, angle):
        self.center_x = float(x)
        self.center_y = float(y)
        self.angle = float(angle)
        self._update_polygon()

def has_collision(trees):
    """Check for collisions between trees."""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def calculate_score(trees):
    """Calculate score for a configuration."""
    all_coords = np.vstack([np.array(t.polygon.exterior.coords) for t in trees])
    min_xy = all_coords.min(axis=0)
    max_xy = all_coords.max(axis=0)
    side = max(max_xy - min_xy)
    return side**2 / len(trees), side

print("ChristmasTree class defined.")

ChristmasTree class defined.


In [3]:
def create_lattice_from_base(base_trees, nt, dx, dy):
    """
    Create a lattice of trees from 2 base trees.
    
    Args:
        base_trees: List of 2 ChristmasTree objects (the base configuration)
        nt: [nx, ny] - number of translations in x and y directions
        dx: Translation step in x direction
        dy: Translation step in y direction
    
    Returns:
        List of ChristmasTree objects forming the lattice
    """
    nx, ny = nt
    trees = []
    
    for i in range(nx):
        for j in range(ny):
            for base in base_trees:
                x, y, angle = base.get_params()
                new_x = x + i * dx
                new_y = y + j * dy
                trees.append(ChristmasTree(new_x, new_y, angle))
    
    return trees

# Test with the egortrushin initial configuration
# Base trees from the kernel: [-2.93, -4.25, 67°] and [-3.93, -4.17, 250°]
base_tree1 = ChristmasTree(-2.93069232, -4.24856960, 67)
base_tree2 = ChristmasTree(-3.92971914, -4.16631769, 250)
base_trees = [base_tree1, base_tree2]

print("Base trees:")
for i, t in enumerate(base_trees):
    print(f"  Tree {i}: x={t.center_x:.4f}, y={t.center_y:.4f}, angle={t.angle:.2f}")

Base trees:
  Tree 0: x=-2.9307, y=-4.2486, angle=67.00
  Tree 1: x=-3.9297, y=-4.1663, angle=250.00


In [4]:
# Test creating a lattice for N=72 (4x9 grid of 2-tree units = 72 trees)
nt = [4, 9]  # 4 translations in x, 9 in y -> 4*9*2 = 72 trees
dx = 1.0  # Initial guess for translation step
dy = 1.0

trees_72 = create_lattice_from_base(base_trees, nt, dx, dy)
print(f"Created {len(trees_72)} trees for N=72")

# Check for collisions
if has_collision(trees_72):
    print("WARNING: Collisions detected!")
else:
    print("No collisions.")

# Calculate score
score, side = calculate_score(trees_72)
print(f"Score: {score:.6f}, Side: {side:.4f}")

Created 72 trees for N=72
Score: 1.059318, Side: 8.7333


In [5]:
# Load baseline scores for comparison
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

baseline_df['x_val'] = baseline_df['x'].apply(parse_value)
baseline_df['y_val'] = baseline_df['y'].apply(parse_value)
baseline_df['deg_val'] = baseline_df['deg'].apply(parse_value)
baseline_df['n'] = baseline_df['id'].apply(lambda x: int(x.split('_')[0]))

# Get baseline scores for each N
baseline_scores = {}
for n in range(1, 201):
    group = baseline_df[baseline_df['n'] == n]
    trees_data = [(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    trees = [ChristmasTree(x, y, deg) for x, y, deg in trees_data]
    score, side = calculate_score(trees)
    baseline_scores[n] = {'score': score, 'side': side}

print(f"Baseline total score: {sum(s['score'] for s in baseline_scores.values()):.6f}")
print(f"\nBaseline scores for target N values:")
for n in [36, 72, 100, 144, 156, 196, 200]:
    print(f"  N={n}: score={baseline_scores[n]['score']:.6f}, side={baseline_scores[n]['side']:.4f}")

Baseline total score: 70.676102

Baseline scores for target N values:
  N=36: score=0.358820, side=3.5941
  N=72: score=0.348559, side=5.0096
  N=100: score=0.345531, side=5.8782
  N=144: score=0.342276, side=7.0205
  N=156: score=0.329987, side=7.1748
  N=196: score=0.333299, side=8.0825
  N=200: score=0.337731, side=8.2187


In [6]:
# Implement Simulated Annealing to optimize the lattice configuration
# We optimize: base tree positions, base tree angles, dx, dy

class LatticeOptimizer:
    def __init__(self, n_target, nt, max_iter=1000, T_start=0.01, T_end=0.0001):
        self.n_target = n_target
        self.nt = nt
        self.max_iter = max_iter
        self.T_start = T_start
        self.T_end = T_end
        
    def optimize(self, initial_base_trees, initial_dx, initial_dy):
        """Optimize the lattice configuration using SA."""
        # Current state
        base_trees = [ChristmasTree(*t.get_params()) for t in initial_base_trees]
        dx, dy = initial_dx, initial_dy
        
        # Create initial lattice
        trees = create_lattice_from_base(base_trees, self.nt, dx, dy)
        if len(trees) < self.n_target:
            print(f"Warning: lattice has {len(trees)} trees, need {self.n_target}")
            return None, float('inf')
        
        trees = trees[:self.n_target]  # Take only n_target trees
        
        if has_collision(trees):
            # Try to find non-colliding initial config
            for scale in [1.2, 1.5, 2.0, 2.5, 3.0]:
                trees = create_lattice_from_base(base_trees, self.nt, dx*scale, dy*scale)
                trees = trees[:self.n_target]
                if not has_collision(trees):
                    dx, dy = dx*scale, dy*scale
                    break
            else:
                print("Could not find non-colliding initial config")
                return None, float('inf')
        
        best_score, _ = calculate_score(trees)
        best_state = (copy.deepcopy(base_trees), dx, dy)
        current_score = best_score
        
        # SA loop
        for i in range(self.max_iter):
            T = self.T_start * (self.T_end / self.T_start) ** (i / self.max_iter)
            
            # Perturb state
            new_base_trees = [ChristmasTree(*t.get_params()) for t in base_trees]
            new_dx, new_dy = dx, dy
            
            move_type = random.choice(['pos', 'angle', 'dx', 'dy'])
            if move_type == 'pos':
                idx = random.randint(0, 1)
                x, y, angle = new_base_trees[idx].get_params()
                x += random.uniform(-0.1, 0.1)
                y += random.uniform(-0.1, 0.1)
                new_base_trees[idx].set_params(x, y, angle)
            elif move_type == 'angle':
                idx = random.randint(0, 1)
                x, y, angle = new_base_trees[idx].get_params()
                angle += random.uniform(-10, 10)
                new_base_trees[idx].set_params(x, y, angle)
            elif move_type == 'dx':
                new_dx += random.uniform(-0.05, 0.05)
            else:
                new_dy += random.uniform(-0.05, 0.05)
            
            # Create new lattice
            new_trees = create_lattice_from_base(new_base_trees, self.nt, new_dx, new_dy)
            new_trees = new_trees[:self.n_target]
            
            if has_collision(new_trees):
                continue
            
            new_score, _ = calculate_score(new_trees)
            
            # Accept or reject
            delta = new_score - current_score
            if delta < 0 or random.random() < np.exp(-delta / T):
                base_trees = new_base_trees
                dx, dy = new_dx, new_dy
                current_score = new_score
                
                if new_score < best_score:
                    best_score = new_score
                    best_state = (copy.deepcopy(base_trees), dx, dy)
        
        return best_state, best_score

print("LatticeOptimizer defined.")

LatticeOptimizer defined.


In [7]:
# Test optimization for N=72
print("Optimizing N=72...")
optimizer = LatticeOptimizer(n_target=72, nt=[4, 9], max_iter=2000)
best_state, best_score = optimizer.optimize(base_trees, 1.0, 1.0)

if best_state:
    print(f"Best score for N=72: {best_score:.6f}")
    print(f"Baseline score for N=72: {baseline_scores[72]['score']:.6f}")
    print(f"Improvement: {baseline_scores[72]['score'] - best_score:.6f}")
else:
    print("Optimization failed.")

Optimizing N=72...


Best score for N=72: 0.390970
Baseline score for N=72: 0.348559
Improvement: -0.042411


In [ ]:
# Test on multiple N values
results = {}

# N values that can be formed as 2 * nx * ny
test_configs = [
    (36, [3, 6]),   # 3*6*2 = 36
    (72, [4, 9]),   # 4*9*2 = 72
    (100, [5, 10]), # 5*10*2 = 100
    (144, [6, 12]), # 6*12*2 = 144
    (156, [6, 13]), # 6*13*2 = 156
    (196, [7, 14]), # 7*14*2 = 196
    (200, [5, 20]), # 5*20*2 = 200
]

for n, nt in test_configs:
    print(f"\nOptimizing N={n} with nt={nt}...")
    optimizer = LatticeOptimizer(n_target=n, nt=nt, max_iter=3000)
    best_state, best_score = optimizer.optimize(base_trees, 1.0, 1.0)
    
    if best_state:
        results[n] = {
            'score': best_score,
            'state': best_state,
            'baseline': baseline_scores[n]['score'],
            'improvement': baseline_scores[n]['score'] - best_score
        }
        print(f"  Best: {best_score:.6f}, Baseline: {baseline_scores[n]['score']:.6f}, Improvement: {results[n]['improvement']:.6f}")
    else:
        print(f"  Optimization failed.")

In [ ]:
# Summary of results
print("\n" + "="*60)
print("SUMMARY OF LATTICE OPTIMIZATION RESULTS")
print("="*60)

total_improvement = 0
for n, res in sorted(results.items()):
    status = "BETTER" if res['improvement'] > 0 else "WORSE"
    print(f"N={n:3d}: Lattice={res['score']:.6f}, Baseline={res['baseline']:.6f}, Diff={res['improvement']:+.6f} [{status}]")
    if res['improvement'] > 0:
        total_improvement += res['improvement']

print(f"\nTotal potential improvement: {total_improvement:.6f}")
print(f"Baseline total score: {sum(s['score'] for s in baseline_scores.values()):.6f}")

In [ ]:
# The lattice approach is likely not beating the highly optimized baseline
# Let's check if any N values show improvement

improved_n = [n for n, res in results.items() if res['improvement'] > 0]
print(f"N values with improvement: {improved_n}")

if not improved_n:
    print("\nNo improvement found with lattice approach.")
    print("The baseline is already highly optimized.")
    print("\nFalling back to baseline submission.")

In [ ]:
# Save metrics and create submission
import json
import shutil

# Calculate final score (use baseline since no improvement)
final_score = sum(s['score'] for s in baseline_scores.values())

metrics = {
    'cv_score': final_score,
    'baseline_score': 70.676102,
    'lattice_results': {str(n): {'score': res['score'], 'baseline': res['baseline'], 'improvement': res['improvement']} 
                        for n, res in results.items()},
    'improved_n_values': improved_n,
    'notes': 'Egortrushin lattice approach tested on multiple N values. No improvement over baseline found.'
}

with open('/home/code/experiments/003_egortrushin_lattice/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

# Copy baseline submission since no improvement
shutil.copy('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv', 
            '/home/submission/submission.csv')

print(f"Metrics saved.")
print(f"Final score: {final_score:.6f}")

In [ ]:
# Submission cell
class EgortrushinLattice:
    """Egortrushin 2-tree base configuration lattice approach."""
    def __init__(self, data='single'):
        self.data = data
    
    def get_submission(self):
        return '/home/submission/submission.csv'

model = EgortrushinLattice(data='single')
print(f"Model class: {model.__class__.__name__}")